In [ ]:
# System
import sys
import os
import json

# Misc
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch


# ML
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

from shapely.geometry import shape

# Augmentations
import albumentations as A

# Custom
sys.path.append(os.path.abspath(".."))
from hover_net.dataloader.dataset import get_dataloader
from hover_net.datasets.puma_dataset import PumaDataset

In [ ]:
IMAGE_PATH      = '../data/01_training_dataset_tif_ROIs'
GEOJSON_PATH    = '../data/01_training_dataset_geojson_nuclei'
PATCH_SIZE      = 512
BATCH_SIZE      = 1

dataset = PumaDataset(
    image_path=IMAGE_PATH,
    geojson_path=GEOJSON_PATH,
    input_shape=(
        PATCH_SIZE,
        PATCH_SIZE
    ),
    mask_shape=(
        PATCH_SIZE,
        PATCH_SIZE
    ),
    run_mode="test",
    augment=True
)

img, ann = PumaDataset.load_data(dataset, 7)
print("Pre-augmentation")
print(f"Image shape: {img.shape}")
print(f"Annotation shape: {ann.shape}")
aug = A.Compose([A.CenterCrop(height=PATCH_SIZE, width=PATCH_SIZE)])
augmented = aug(image=img, mask=ann)
img = augmented["image"]
ann = augmented["mask"]

# Define the colors for each class
colors = ['white', 'teal', 'lightblue', 'lightgreen']  # Example colors for 0, 1, 2, 3
class_labels = ['Tumor', 'TILs', 'Other']  # Labels for each color
cmap = ListedColormap(colors)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img, interpolation='none')

ann_image = ann[:, :, 1]
#ax[1].imshow(ann_image, cmap=cmap)
contour_fill = ax[1].contourf(
    ann_image, 
    levels=[-0.5, 0.5, 1.5, 2.5, 3.5], 
    cmap=cmap, 
    alpha=0.8, 
    extent=(0, ann.shape[1], 0, ann.shape[0])
)
contour_lines = ax[1].contour(
    ann_image, 
    levels=[-0.5, 0.5, 1.5, 2.5, 3.5], 
    cmap=cmap, 
    linewidths=2,
    alpha=1, 
    extent=(0, ann.shape[1], 0, ann.shape[0])
)
ax[1].invert_yaxis()

ax[0].set_title("Image", fontsize=16)
ax[1].set_title("Ground Truth", fontsize=16)

ax[0].tick_params(
    axis='both',  # Apply to both x and y axes
    which='both',  # Apply to major and minor ticks
    bottom=False,  # Remove ticks at the bottom
    left=False,    # Remove ticks at the left
    labelbottom=False,  # Remove tick labels at the bottom
    labelleft=False     # Remove tick labels at the left
)

ax[1].tick_params(
    axis='both',  # Apply to both x and y axes
    which='both',  # Apply to major and minor ticks
    bottom=False,  # Remove ticks at the bottom
    left=False,    # Remove ticks at the left
    labelbottom=False,  # Remove tick labels at the bottom
    labelleft=False     # Remove tick labels at the left
)

ax[1].set_aspect('equal', 'box')

legend_elements = [Patch(facecolor=colors[i+1], label=class_labels[i]) for i in range(len(colors)-1)]
fig.legend(
    handles=legend_elements, 
    loc='lower center', 
    ncol=3,
    fontsize=16,
    bbox_to_anchor=(0.5, -0.1),  # Adjust this value to move the legend up or down
    frameon=True
)

# plt.subplots_adjust(top=0.3)

fig.tight_layout()

fig.savefig("../figures/dataset_example.pdf", bbox_inches='tight')

In [ ]:
GEOJSON_PATH = '../data/01_training_dataset_geojson_nuclei'
geojsons = os.listdir(GEOJSON_PATH)

nuclei_count = {
    "primary":
    {
        "tumor": 0,
        "tils": 0,
        "other": 0
    },
    "metastatic":
    {
        "tumor": 0,
        "tils": 0,
        "other": 0
    }
}

for file in geojsons:
    with open(os.path.join(GEOJSON_PATH, file), encoding="utf-8") as f:
        geojson = json.load(f)
    for feature in geojson["features"]:
        geometry = shape(feature["geometry"])
        label = feature["properties"]["classification"]["name"]

        if geometry.geom_type == "Polygon":
            coords = geometry.exterior.coords
            if label == "nuclei_tumor":
                if "metastatic" in file:
                    nuclei_count["metastatic"]["tumor"] += 1
                else:
                    nuclei_count["primary"]["tumor"] += 1
            elif label in ["nuclei_lymphocyte", "nuclei_plasma_cell"]:
                if "metastatic" in file:
                    nuclei_count["metastatic"]["tils"] += 1
                else:
                    nuclei_count["primary"]["tils"] += 1
            else:
                if "metastatic" in file:
                    nuclei_count["metastatic"]["other"] += 1
                else:
                    nuclei_count["primary"]["other"] += 1

In [ ]:
# Data preparation
categories = ['tumor', 'tils', 'other']
titles = ['Tumor', 'TILs', 'Other']
primary_counts = [nuclei_count["primary"][category] for category in categories]
metastatic_counts = [nuclei_count["metastatic"][category] for category in categories]

# Data preparation for pie chart
primary_sizes = [nuclei_count["primary"][category] for category in categories]
metastatic_sizes = [nuclei_count["metastatic"][category] for category in categories]

# Define consistent colors for both pie charts
colors = ['teal', 'lightblue', 'lightgreen']

# Pie chart for Primary
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Primary pie chart
ax[0].pie(primary_sizes, autopct='%1.1f%%', startangle=90, colors=colors)
ax[0].set_title('Primary Nuclei Distribution', fontsize=16, y=0.92)

# Metastatic pie chart
ax[1].pie(metastatic_sizes, autopct='%1.1f%%', startangle=90, colors=colors)
ax[1].set_title('Metastatic Nuclei Distribution', fontsize=16, y=0.92)

# Adding a single legend at the bottom center
fig.legend(titles, loc="lower center", fontsize=16, ncol=3, bbox_to_anchor=(0.5, 0))

# Display the plot
fig.tight_layout()

fig.savefig("../figures/dataset_distribution.pdf", bbox_inches='tight')

In [ ]:
total_primary_nuclei = sum(nuclei_count['primary'].values())
total_metastatic_nuclei = sum(nuclei_count['metastatic'].values())
print(f"Total primary nuclei: {total_primary_nuclei}")
print(f"Total metastatic nuclei: {total_metastatic_nuclei}")
print(f"Total: {total_primary_nuclei + total_metastatic_nuclei}")

total_tumor = nuclei_count["primary"]["tumor"] + nuclei_count["metastatic"]["tumor"]
total_tils = nuclei_count["primary"]["tils"] + nuclei_count["metastatic"]["tils"]
total_other = nuclei_count["primary"]["other"] + nuclei_count["metastatic"]["other"]

print(f"Total tumor: {total_tumor}")
print(f"Total TILs: {total_tils}")
print(f"Total other: {total_other}")

In [ ]:
# Packages
import numpy as np
from scipy.ndimage import measurements

def get_bounding_box(img):
    """Get bounding box coordinate information."""
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    # due to python indexing, need to add 1 to max
    # else accessing will be 1px in the box, not out
    rmax += 1
    cmax += 1
    return [rmin, rmax, cmin, cmax]


def fix_mirror_padding(ann):
    """
    Deal with duplicated instances due to mirroring in interpolation
    during shape augmentation (scale, rotation etc.).
    """
    current_max_id = np.amax(ann)
    inst_list = list(np.unique(ann))
    inst_list.remove(0)  # 0 is background
    for inst_id in inst_list:
        inst_map = np.array(ann == inst_id, np.uint8)
        remapped_ids = measurements.label(inst_map)[0]
        remapped_ids[remapped_ids > 1] += current_max_id
        ann[remapped_ids > 1] = remapped_ids[remapped_ids > 1]
        current_max_id = np.amax(ann)
    return ann


def gen_instance_hv_map(ann):
    """
    Generate the HoVer maps for each nuclear instance.

    Args:
        - ann: instance ID map
    Returns:
        - hv_map: horizontal and vertical distance maps
    """
    fixed_ann = fix_mirror_padding(ann)

    x_map = np.zeros(ann.shape[:2], dtype=np.float32)
    y_map = np.zeros(ann.shape[:2], dtype=np.float32)

    inst_list = list(np.unique(ann))
    inst_list.remove(0)  # 0 is background
    for inst_id in inst_list:
        inst_map = np.array(fixed_ann == inst_id, np.uint8)
        inst_box = get_bounding_box(inst_map)

        # expand the box by 2px
        # Because we first pad the ann at line 207, the bboxes
        # will remain valid after expansion
        inst_box[0] -= 2
        inst_box[2] -= 2
        inst_box[1] += 2
        inst_box[3] += 2

        inst_map = inst_map[inst_box[0]:inst_box[1], inst_box[2]:inst_box[3]]

        if inst_map.shape[0] < 2 or inst_map.shape[1] < 2:
            continue

        # instance center of mass, rounded to nearest pixel
        inst_com = list(measurements.center_of_mass(inst_map))

        inst_com[0] = int(inst_com[0] + 0.5)
        inst_com[1] = int(inst_com[1] + 0.5)

        inst_x_range = np.arange(1, inst_map.shape[1] + 1)
        inst_y_range = np.arange(1, inst_map.shape[0] + 1)
        # shifting center of pixels grid to instance center of mass
        inst_x_range -= inst_com[1]
        inst_y_range -= inst_com[0]

        inst_x, inst_y = np.meshgrid(inst_x_range, inst_y_range)

        # remove coord outside of instance
        inst_x[inst_map == 0] = 0
        inst_y[inst_map == 0] = 0
        inst_x = inst_x.astype("float32")
        inst_y = inst_y.astype("float32")

        # normalize min into -1 scale
        if np.min(inst_x) < 0:
            inst_x[inst_x < 0] /= -np.amin(inst_x[inst_x < 0])
        if np.min(inst_y) < 0:
            inst_y[inst_y < 0] /= -np.amin(inst_y[inst_y < 0])
        # normalize max into +1 scale
        if np.max(inst_x) > 0:
            inst_x[inst_x > 0] /= np.amax(inst_x[inst_x > 0])
        if np.max(inst_y) > 0:
            inst_y[inst_y > 0] /= np.amax(inst_y[inst_y > 0])

        ####
        x_map_box = x_map[inst_box[0]:inst_box[1], inst_box[2]:inst_box[3]]
        x_map_box[inst_map > 0] = inst_x[inst_map > 0]

        y_map_box = y_map[inst_box[0]:inst_box[1], inst_box[2]:inst_box[3]]
        y_map_box[inst_map > 0] = inst_y[inst_map > 0]

    hv_map = np.dstack([x_map, y_map])
    return hv_map


def gen_targets(ann):
    """
    Generate the targets for the network.
    
    Args:
        - ann: instance ID map
    Returns:
        - target_dict: dictionary containing the hv_maps (X, Y, 2) and np_map (X, Y)
    """
    hv_map = gen_instance_hv_map(ann)
    np_map = ann.copy()
    np_map[np_map > 0] = 1

    target_dict = {
        "hv_map": hv_map,
        "np_map": np_map,
    }

    return target_dict

In [ ]:
IMAGE_PATH      = '../data/01_training_dataset_tif_ROIs'
GEOJSON_PATH    = '../data/01_training_dataset_geojson_nuclei'
PATCH_SIZE      = 512
BATCH_SIZE      = 1

dataset = PumaDataset(
    image_path=IMAGE_PATH,
    geojson_path=GEOJSON_PATH,
    input_shape=(
        PATCH_SIZE,
        PATCH_SIZE
    ),
    mask_shape=(
        PATCH_SIZE,
        PATCH_SIZE
    ),
    run_mode="test",
    augment=True
)

# Get a batch of data
img, ann = PumaDataset.load_data(dataset, 7)
print("Pre-augmentation")
print(f"Image shape: {img.shape}")
print(f"Annotation shape: {ann.shape}")

aug = A.Compose([A.CenterCrop(height=PATCH_SIZE, width=PATCH_SIZE)])
augmented = aug(image=img, mask=ann)
img = augmented["image"]
ann = augmented["mask"]
print("Post-augmentation")
print(f"Image shape: {img.shape}")
print(f"Annotation shape: {ann.shape}")

targets = gen_targets(ann[..., 1])
print(f"Target keys: {targets.keys()}")
print(f"Horizontal and vertical map shape: {targets['hv_map'].shape}")
print(f"Nuclei pixel map shape: {targets['np_map'].shape}")

fig, ax = plt.subplots(1, 5, figsize=(15, 5))
ax[0].imshow(img)
ax[0].set_title("Image")
ax[1].imshow(targets["hv_map"][..., 0])
ax[1].set_title("Horizontal map")
ax[2].imshow(targets["hv_map"][..., 1])
ax[2].set_title("Vertical map")
ax[3].imshow(targets["np_map"])
ax[3].set_title("Nuclei pixel map")
ax[4].imshow(ann[..., 1])

In [ ]:
IMAGE_PATH      = '../data/01_training_dataset_tif_ROIs'
GEOJSON_PATH    = '../data/01_training_dataset_geojson_nuclei'
PATCH_SIZE      = 512
BATCH_SIZE      = 1

dataset = PumaDataset(
    image_path=IMAGE_PATH,
    geojson_path=GEOJSON_PATH,
    input_shape=(
        PATCH_SIZE,
        PATCH_SIZE
    ),
    mask_shape=(
        PATCH_SIZE,
        PATCH_SIZE
    ),
    run_mode="test",
    augment=True
)

img, ann = PumaDataset.load_data(dataset, 7)
print("Pre-augmentation")
print(f"Image shape: {img.shape}")
print(f"Annotation shape: {ann.shape}")
aug = A.Compose([A.CenterCrop(height=PATCH_SIZE, width=PATCH_SIZE)])
augmented = aug(image=img, mask=ann)
img = augmented["image"]
ann = augmented["mask"]

# Define the colors for each class
colors = ['white', 'teal', 'lightblue', 'lightgreen']  # Example colors for 0, 1, 2, 3
class_labels = ['Tumor', 'TILs', 'Other']  # Labels for each color
cmap = ListedColormap(colors)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ann_image = ann[:, :, 1]
contour_fill = ax.contourf(
    ann_image, 
    levels=[-0.5, 0.5, 1.5, 2.5, 3.5], 
    cmap=cmap, 
    alpha=0.8, 
    extent=(0, ann.shape[1], 0, ann.shape[0])
)
contour_lines = ax.contour(
    ann_image, 
    levels=[-0.5, 0.5, 1.5, 2.5, 3.5], 
    cmap=cmap, 
    linewidths=2,
    alpha=1, 
    extent=(0, ann.shape[1], 0, ann.shape[0])
)
ax.invert_yaxis()
ax.set_title("Type Map", fontsize=16)

ax.tick_params(
    axis='both',  # Apply to both x and y axes
    which='both',  # Apply to major and minor ticks
    bottom=False,  # Remove ticks at the bottom
    left=False,    # Remove ticks at the left
    labelbottom=False,  # Remove tick labels at the bottom
    labelleft=False     # Remove tick labels at the left
)

ax.set_aspect('equal', 'box')
fig.tight_layout()

fig.savefig("../figures/data_tp.png", bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(img)

ax.set_title("Image", fontsize=16)

ax.tick_params(
    axis='both',  # Apply to both x and y axes
    which='both',  # Apply to major and minor ticks
    bottom=False,  # Remove ticks at the bottom
    left=False,    # Remove ticks at the left
    labelbottom=False,  # Remove tick labels at the bottom
    labelleft=False     # Remove tick labels at the left
)

ax.set_aspect('equal', 'box')
fig.tight_layout()

fig.savefig("../figures/data_img.png", bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.imshow(targets["hv_map"][..., 0], cmap='gist_rainbow')

ax.set_title("Horizontal Map", fontsize=16)

ax.tick_params(
    axis='both',  # Apply to both x and y axes
    which='both',  # Apply to major and minor ticks
    bottom=False,  # Remove ticks at the bottom
    left=False,    # Remove ticks at the left
    labelbottom=False,  # Remove tick labels at the bottom
    labelleft=False     # Remove tick labels at the left
)

ax.set_aspect('equal', 'box')
fig.tight_layout()

fig.savefig("../figures/data_h.png", bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.imshow(targets["hv_map"][..., 1], cmap='gist_rainbow')

ax.set_title("Vertical Map", fontsize=16)

ax.tick_params(
    axis='both',  # Apply to both x and y axes
    which='both',  # Apply to major and minor ticks
    bottom=False,  # Remove ticks at the bottom
    left=False,    # Remove ticks at the left
    labelbottom=False,  # Remove tick labels at the bottom
    labelleft=False     # Remove tick labels at the left
)

ax.set_aspect('equal', 'box')
fig.tight_layout()

fig.savefig("../figures/data_v.png", bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

colors = ['white', 'red']  # Example colors for 0, 1, 2, 3
cmap = ListedColormap(colors)

ax.imshow(targets["np_map"], cmap=cmap)

ax.set_title("Nuclear Pixel Map", fontsize=16)

ax.tick_params(
    axis='both',  # Apply to both x and y axes
    which='both',  # Apply to major and minor ticks
    bottom=False,  # Remove ticks at the bottom
    left=False,    # Remove ticks at the left
    labelbottom=False,  # Remove tick labels at the bottom
    labelleft=False     # Remove tick labels at the left
)

ax.set_aspect('equal', 'box')
fig.tight_layout()

fig.savefig("../figures/data_np.png", bbox_inches='tight', dpi=300)